<a href="https://colab.research.google.com/github/nguyenngoctruong2k1/CS114.L22.KHCL/blob/main/Tensorflow/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# BÀI TOÁN PHÂN LOẠI VĂN BẢN CƠ BẢN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Hướng dẫn này trình bày phân loại văn bản bắt đầu từ các tệp văn bản thuần túy được lưu trữ trên đĩa.  Bạn sẽ train một bộ phân loại nhị phân để thực hiện phân tích tình cảm trên tập dữ liệu IMDB.  Ở cuối sổ tay, có một bài tập để bạn thử, trong đó bạn sẽ huấn luyện một bộ phân loại đa lớp để dự đoán thẻ cho một câu hỏi lập trình trên Stack Overflow.


In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# gọi thư viện

In [ ]:
print(tf.__version__)

2.4.1


## Phân tích tình cảm

Notebook này đào tạo một mô hình phân tích tình cảm để phân loại các bài phê bình phim thành *positive* or *negative*, based on the text of the review. Đây là một ví dụ về *binary* hoặc *two-class—classification*, một loại vấn đề học máy quan trọng và có thể áp dụng rộng rãi.

You'll use the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are *balanced*, meaning they contain an equal number of positive and negative reviews.


### Lấy và tìm hiểu dữ liệu

#### Lấy dữ liệu

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 3s 0us/step


#### Vọc với dữ liệu đã lấy

In [ ]:
os.listdir(dataset_dir)

['imdb.vocab', 'train', 'test', 'imdbEr.txt', 'README']

In [ ]:
train_dir = os.path.join(dataset_dir, 'train') 
# Giống như lệnh cd ..
os.listdir(train_dir)

['unsup',
 'urls_neg.txt',
 'labeledBow.feat',
 'urls_pos.txt',
 'neg',
 'pos',
 'unsupBow.feat',
 'urls_unsup.txt']

`aclImdb/train/pos` và `aclImdb/train/neg` thư mục chứa nhiều tệp văn bản, mỗi tệp là một bài đánh giá phim duy nhất.  Chúng ta hãy nhìn vào một trong số đó.

In [ ]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


### Tải tập dữ liệu

Tiếp theo, bạn sẽ tải dữ liệu ra đĩa và chuẩn bị nó thành một định dạng phù hợp để train.  Để làm như vậy, bạn sẽ sử dụng [text_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text_dataset_from_directory) utility, và nó mong đợi được một câu trúc như sau:
```
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
```

Để chuẩn bị một tập dữ liệu cho phân loại nhị phân, bạn sẽ cần hai thư mục trên đĩa, tương ứng với `class_a` và `class_b`. Đây sẽ là những đánh giá tích cực và tiêu cực về phim, có thể tìm thấy trong  `aclImdb/train/pos` và `aclImdb/train/neg`. Vì tập dữ liệu IMDB chứa các thư mục bổ sung, bạn sẽ xóa chúng trước khi sử dụng tiện ích này.

In [ ]:
# Xóa thư mục unsup trong thư mục train
import shutil
remove_dir = os.path.join(train_dir, 'unsup')
print(remove_dir)
shutil.rmtree(remove_dir, ignore_errors=True)

./aclImdb/train/unsup


- Tiếp theo, sử dụng `text_dataset_from_directory` là hàm để tạo `tf.data.Dataset`. [tf.data](https://www.tensorflow.org/guide/data) là một tập hợp các công cụ mạnh mẽ làm việc với dữ liệu.

- Khi chạy thử một chương trình máy học, cách tốt nhất là chia dữ liệu thành 3 phần: [train](https://developers.google.com/machine-learning/glossary#training_set), [validation](https://developers.google.com/machine-learning/glossary#validation_set), và [test](https://developers.google.com/machine-learning/glossary#test-set). 

- Tập dữ liệu IMDB đã được chia thành huấn luyện và kiểm tra, nhưng nó thiếu tập hợp xác thực.  Hãy tạo một tập hợp xác thực bằng cách sử dụng phân tách 80:20 của dữ liệu huấn luyện bằng cách sử dụng đối số validation_split bên dưới.

In [ ]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)
# print(raw_train_ds)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>


Như bạn có thể thấy ở trên, có 25,000 ví dụ trong thư mục đào tạo, trong đó bạn sẽ sử dụng 80% (hoặc 20.000) để đào tạo.  Như bạn sẽ thấy trong giây lát, bạn có thể đào tạo một mô hình bằng cách chuyển trực tiếp tập dữ liệu tới `model.fit`. Nếu bạn là người mới `tf.data`, bạn cũng có thể lặp lại tập dữ liệu và in ra một vài ví dụ như sau.

In [ ]:
for text_batch, label_batch in raw_train_ds.take(2):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])
print(raw_train_ds.take(2))

Review b'Belmondo is a tough cop. He goes after a big-time drug dealer (played by Henry Silva, normally a great villain - see "Sharky\'s Machine"; but here he is clearly dubbed, and because of that he lacks his usual charisma). He goes to the scuzziest places of Paris and Marseilles, asks for some names, beats up some people, gets the names, goes to more scuzzy places, asks for more names, beats up more people, etc. The whole movie is punch after punch after punch. It seems that the people who made it had no other ambition than to create the French equivalent of "Dirty Harry". Belmondo, who was 50 here, does perform some good stunts at the beginning; apart from those, "Le Marginal" is a violent, episodic, trite, shallow and forgettable cop movie. (*1/2)'
Label 0
Review b'Wow. The only people reviewing this positively are the Carpenter apologists. I know a lot of those. The guys that\'ll watch John Carpenter squat on celluloid and pinch out a movie and proclaim it a masterwork of horror

Lưu ý rằng các bài đánh giá có chứa văn bản thô (với dấu chấm câu và các thẻ HTML không thường xuyên như `<br/>`).  Bạn sẽ chỉ ra cách xử lý những điều này trong phần sau.

Các nhãn là 0 hoặc 1. Để xem nhãn nào trong số này tương ứng với các đánh giá tích cực và tiêu cực về phim, bạn có thể kiểm tra thuộc tính `class_names` trên tập dữ liệu.


In [ ]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


Tiếp theo, bạn sẽ tạo một tập dữ liệu xác nhận và kiểm tra.  Bạn sẽ sử dụng 5.000 đánh giá còn lại từ tập huấn luyện để xác nhận.

Lưu ý: Khi sử dụng các đối số `validation_split` và `subset`, hãy đảm bảo chỉ định một hạt ngẫu nhiên hoặc chuyển `shuffle = False`, để phân tách xác thực và đào tạo không có sự chồng chéo.

In [ ]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


Note: The Preprocessing APIs used in the following section are experimental in TensorFlow 2.3 and subject to change.

### Chuẩn hóa dữ liệu cho việc training

Tiếp theo, bạn sẽ chuẩn hóa, mã hóa và vectơ hóa dữ liệu bằng cách sử dụng lớp `preprocessing.TextVectorization`. 

Tiêu chuẩn hóa đề cập đến việc xử lý trước văn bản, thường là loại bỏ dấu chấm câu hoặc các phần tử HTML để đơn giản hóa tập dữ liệu.  Tokenization đề cập đến việc chia nhỏ các chuỗi thành các mã thông báo (ví dụ: tách một câu thành các từ riêng lẻ, bằng cách tách trên khoảng trắng).  Vectơ hóa đề cập đến việc chuyển đổi các mã thông báo thành số để chúng có thể được đưa vào mạng nơ-ron.  Tất cả các nhiệm vụ này có thể được thực hiện với lớp này.

Như bạn đã thấy ở trên, các bài đánh giá chứa các thẻ HTML khác nhau như `<br />`. Các thẻ này sẽ không bị xóa bởi trình chuẩn hóa mặc định trong lớp `TextVectorization` (chuyển đổi văn bản thành chữ thường và loại bỏ dấu chấm câu theo mặc định, nhưng không loại bỏ HTML).  Bạn sẽ viết một hàm tiêu chuẩn hóa tùy chỉnh để loại bỏ HTML.

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

Tiếp theo, bạn sẽ tạo một lớp `TextVectorization`. Bạn sẽ sử dụng lớp này để chuẩn hóa, tokenize và vectơ hóa dữ liệu của chúng tôi. Bạn đặt `output_mode` thành `int` để tạo các chỉ số số nguyên duy nhất cho mỗi mã thông báo.

Lưu ý rằng bạn đang sử dụng chức năng phân tách mặc định và chức năng tiêu chuẩn hóa tùy chỉnh mà bạn đã xác định ở trên.  Bạn cũng sẽ xác định một số hằng số cho mô hình, chẳng hạn như giá trị tối đa rõ ràng `sequence_length`, điều này sẽ khiến lớp đệm hoặc cắt ngắn các trình tự cho chính xác giá trị `sequence_length`.

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Tiếp theo, bạn sẽ gọi `adapt` để phù hợp với trạng thái của lớp tiền xử lý với tập dữ liệu.  Điều này sẽ làm cho mô hình xây dựng một chỉ mục của chuỗi thành số nguyên.

Lưu ý: điều quan trọng là chỉ sử dụng dữ liệu đào tạo của bạn khi gọi thích ứng (sử dụng bộ thử nghiệm sẽ làm rò rỉ thông tin).

In [ ]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Hãy tạo một hàm để xem kết quả của việc sử dụng lớp này để xử lý trước một số dữ liệu.

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Như bạn có thể thấy ở trên, mỗi mã thông báo đã được thay thế bằng một số nguyên.  Bạn có thể tra cứu mã thông báo (chuỗi) mà mỗi số nguyên tương ứng bằng cách gọi `.get_vocabulary ()` trên lớp.

In [ ]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

Bạn gần như đã sẵn sàng để đào tạo mô hình của mình.  Là bước tiền xử lý cuối cùng, bạn sẽ áp dụng lớp TextVectorization mà bạn đã tạo trước đó cho tập dữ liệu huấn luyện, xác thực và thử nghiệm.

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Định cấu hình tập dữ liệu cho hiệu suất

Đây là hai phương pháp quan trọng bạn nên sử dụng khi tải dữ liệu để đảm bảo rằng I / O không bị chặn.

`.cache()` giữ dữ liệu trong bộ nhớ sau khi nó được tải ra khỏi đĩa.  Điều này sẽ đảm bảo tập dữ liệu không trở thành nút cổ chai trong khi đào tạo mô hình của bạn.  Nếu tập dữ liệu của bạn quá lớn để vừa với bộ nhớ, bạn cũng có thể sử dụng phương pháp này để tạo bộ đệm ẩn trên đĩa hiệu quả, hiệu quả hơn để đọc so với nhiều tệp nhỏ.

`.prefetch()` chồng chéo tiền xử lý dữ liệu và thực thi mô hình trong khi đào tạo.

Bạn có thể tìm hiểu thêm về cả hai phương pháp cũng như cách lưu dữ liệu vào bộ nhớ cache vào đĩa trong [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Tạo mô hình

Đã đến lúc tạo mạng nơ-ron của chúng ta:

In [ ]:
embedding_dim = 16

In [ ]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Các lớp được xếp chồng lên nhau tuần tự để xây dựng bộ phân loại:

1. Lớp đầu tiên là một lớp `Embedding`.  Lớp này nhận các đánh giá được mã hóa số nguyên và tìm kiếm một vectơ nhúng cho mỗi từ-chỉ mục.  Các vectơ này được học khi mô hình đào tạo.  Các vectơ thêm một thứ nguyên vào mảng đầu ra.  Các kích thước kết quả là: `(batch, sequence, embedding)`.  Để tìm hiểu thêm về cách nhúng, hãy xem [word embedding tutorial](../text/word_embeddings.ipynb).
2. Tiếp theo, một lớp `GlobalAveragePooling1D` trả về một vectơ đầu ra có độ dài cố định cho mỗi ví dụ bằng cách lấy trung bình trên thứ nguyên trình tự.  Điều này cho phép mô hình xử lý đầu vào có độ dài thay đổi, theo cách đơn giản nhất có thể.
3. Vectơ đầu ra có độ dài cố định này được chuyển qua một kết nối đầy đủ (`Dense`)  lớp với 16 đơn vị ẩn.
4. Lớp cuối cùng được kết nối dày đặc với một nút đầu ra duy nhất.

### Chức năng mất mát và trình tối ưu hóa

Một mô hình cần một chức năng mất mát và một trình tối ưu hóa để đào tạo.  Vì đây là một vấn đề phân loại nhị phân và mô hình xuất ra một xác suất (một lớp đơn vị với kích hoạt sigmoid), bạn sẽ sử dụng BinaryCrossentropy` loss function.

Bây giờ, hãy định cấu hình mô hình để sử dụng an optimizer and a loss function:

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### Train the model

Bạn sẽ huấn luyện mô hình bằng cách chuyển đối tượng tập dữ liệu sang phương thức fit.

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

### Đánh giá mô hình

Hãy xem mô hình hoạt động như thế nào.  Hai giá trị sẽ được trả về.  Mất mát (một con số đại diện cho lỗi của chúng tôi, giá trị càng thấp càng tốt) và độ chính xác.

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Cách tiếp cận khá ngây thơ này đạt độ chính xác khoảng 86%.

### Tạo một biểu đồ về độ chính xác và mất mát theo thời gian


`model.fit()` trả về một `History` chứa một từ điển với mọi thứ đã xảy ra trong quá trình đào tạo:

In [ ]:
history_dict = history.history
history_dict.keys()

Có bốn mục nhập: một mục cho mỗi chỉ số được giám sát trong quá trình đào tạo và xác nhận.  Bạn có thể sử dụng chúng để lập biểu đồ về sự mất mát trong quá trình đào tạo và xác thực để so sánh, cũng như độ chính xác của quá trình đào tạo và xác nhận:

In [ ]:
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

Trong biểu đồ này, các dấu chấm thể hiện sự mất mát trong quá trình huấn luyện và độ chính xác, còn các đường liền nét là sự mất xác thực và độ chính xác.

Lưu ý rằng tổn thất đào tạo giảm theo từng kỷ nguyên và độ chính xác đào tạo tăng lên theo từng kỷ nguyên.  Điều này được mong đợi khi sử dụng tối ưu hóa giảm dần độ dốc — nó sẽ giảm thiểu số lượng mong muốn trên mỗi lần lặp.

Đây không phải là trường hợp của việc mất xác thực và độ chính xác — chúng dường như đạt đến đỉnh điểm trước độ chính xác của quá trình huấn luyện.  Đây là một ví dụ về overfitting: mô hình hoạt động tốt hơn trên dữ liệu đào tạo so với dữ liệu mà nó chưa từng thấy trước đây.  Sau thời điểm này, mô hình tối ưu hóa quá mức và tìm hiểu các đại diện cụ thể cho dữ liệu đào tạo mà không tổng quát hóa để kiểm tra dữ liệu.

Đối với trường hợp cụ thể này, bạn có thể ngăn chặn việc trang bị quá mức bằng cách dừng đào tạo khi độ chính xác xác nhận không còn tăng nữa.  Một cách để làm như vậy là sử dụng `tf.keras.callbacks.EarlyStopping` callback.

## Export the model

Trong đoạn mã trên, bạn đã áp dụng `TextVectorization` lớp vào tập dữ liệu trước khi cung cấp văn bản cho mô hình.  Nếu bạn muốn làm cho mô hình của mình có khả năng xử lý các chuỗi thô (ví dụ: để đơn giản hóa việc triển khai nó), bạn có thể bao gồm `TextVectorization` lớp bên trong mô hình của bạn.  Để làm như vậy, bạn có thể tạo một mô hình mới bằng cách sử dụng trọng lượng bạn vừa tập.

In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

### Suy luận về dữ liệu mới

Để nhận dự đoán cho các ví dụ mới, bạn có thể chỉ cần gọi `model.predict()`.

In [ ]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

export_model.predict(examples)

Bao gồm logic tiền xử lý văn bản bên trong mô hình của bạn cho phép bạn xuất một mô hình để sản xuất giúp đơn giản hóa việc triển khai và giảm khả năng [train/test skew](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew).

Có một sự khác biệt về hiệu suất cần lưu ý khi chọn vị trí áp dụng lớp TextVectorization của bạn.  Sử dụng nó bên ngoài mô hình của bạn cho phép bạn xử lý CPU không đồng bộ và lưu vào bộ đệm dữ liệu của bạn khi đào tạo về GPU.  Vì vậy, nếu bạn đang đào tạo mô hình của mình trên GPU, bạn có thể muốn sử dụng tùy chọn này để có được hiệu suất tốt nhất trong khi phát triển mô hình của mình, sau đó chuyển sang bao gồm lớp `TextVectorization` bên trong mô hình của bạn khi bạn sẵn sàng chuẩn bị triển khai.

Đi đến [tutorial](https://www.tensorflow.org/tutorials/keras/save_and_load) để tìm hiểu thêm về saving models.

## Exercise: multiclass classification on Stack Overflow questions

This tutorial showed how to train a binary classifier from scratch on the IMDB dataset. As an exercise, you can modify this notebook to train a multiclass classifier to predict the tag of a programming question on [Stack Overflow](http://stackoverflow.com/).

We have prepared a [dataset](http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz) for you to use containing the body of several thousand programming questions (for example, "How can sort a dictionary by value in Python?") posted to Stack Overflow. Each of these is labeled with exactly one tag (either Python, CSharp, JavaScript, or Java). Your task is to take a question as input, and predict the appropriate tag, in this case, Python. 

The dataset you will work with contains several thousand questions extracted from the much larger public Stack Overflow dataset on [BigQuery](https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow), which contains more than 17 million posts.

After downloading the dataset, you will find it has a similar directory structure to the IMDB dataset you worked with previously:

```
train/
...python/
......0.txt
......1.txt
...javascript/
......0.txt
......1.txt
...csharp/
......0.txt
......1.txt
...java/
......0.txt
......1.txt
```

Note: to increase the difficulty of the classification problem, we have replaced any occurences of the words Python, CSharp, JavaScript, or Java in the programming questions with the word *blank* (as many questions contain the language they're about). 

To complete this exercise, you should modify this notebook to work with the Stack Overflow dataset by making the following modifications:

1. At the top of your notebook, update the code that downloads the IMDB dataset with code to download the [Stack Overflow dataset](http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz) we have prepreared. As the Stack Overflow dataset has a similar directory structure, you will not need to make many modifications. 

1. Modify the last layer of your model to read `Dense(4)`, as there are now four output classes.

1. When you compile your model, change the loss to `losses.SparseCategoricalCrossentropy`. This is the correct loss function to use for a multiclass classification problem, when the labels for each class are integers (in our case, they can be 0, *1*, *2*, or *3*).

1. Once these changes are complete, you will be able to train a multiclass classifier. 


## Learning more

This tutorial introduced text classification from scratch. To learn more about the text classification workflow in general, we recommend reading [this guide](https://developers.google.com/machine-learning/guides/text-classification/) from Google Developers.
